In [1]:
from utz import *
from IPython.display import HTML, SVG, display

import fiona
import geopandas as gpd
from geopy import distance
import shapefile
from shapely.geometry import Point

import plotly.express as px
import plotly.graph_objects as go

import sys
sys.executable

'/opt/homebrew/Caskroom/mambaforge/base/envs/geopandas/bin/python'

## Mapbox configs

In [2]:
token = open(".mapbox-token").read()
mapbox = dict(mapbox=dict(
    style="dark",
    accesstoken=token,
))

In [3]:
from nj_crashes.crashes import c20
from nj_crashes.sri import get_mp_ll, SRI_DB_PATH, SRI_DB_URL, SRI_DB_TABLE

In [4]:
grove_sri = '09061720__'
grove_crashes = c20[c20.SRI.str.contains(grove_sri)]
grove_crashes

,County Code,Municipality Code,Department Case Number,County Name,Municipality Name,Police Dept Code,Police Department,Police Station,Total Killed,Total Injured,Pedestrians Killed,Pedestrians Injured,Severity,Intersection,Alcohol Involved,HazMat Involved,Crash Type Code,Total Vehicles Involved,Crash Location,Location Direction,Route,Route Suffix,SRI,MP,Road System,Road Character,Road Horizontal Alignment,Road Grade,Road Surface Type,Surface Condition,Light Condition,Environmental Condition,Road Divided By,Temporary Traffic Control Zone,Distance To Cross Street,Unit Of Measurement,Directn From Cross Street,Cross Street Name,Is Ramp,Ramp To/From Route Name,Ramp To/From Route Direction,Posted Speed,Posted Speed Cross Street,First Harmful Event,Latitude,Longitude,Cell Phone In Use Flag,Other Property Damage,Reporting Badge No.,Date
80141,09,06,14178-20,HUDSON,JERSEY CITY,01,JERSEY CITY PD,EAST,0,0,0,0,Property Damage,B,False,False,06,2,GROVE ST / MANILA AVE,,,,09061720__,1.34,07,,01,04,02,01,01,01,01,01,15,FE,N,MONTGOMERY ST,,,,25,25,,NaN,NaN,N,,2934,2020-08-12 17:22:00
80262,09,06,20-000385,HUDSON,JERSEY CITY,01,JERSEY CITY PD,EAST,0,0,0,0,Property Damage,I,False,False,02,2,GROVE ST / MANILA AVE,W,,,09061720__,1.29,07,,01,04,02,01,07,01,04,01,,AT,,MERCER ST,,,,25,25,26,NaN,NaN,N,,3009,2020-01-06 18:43:00
80313,09,06,20-000652,HUDSON,JERSEY CITY,01,JERSEY CITY PD,EAST DISTRICT,0,0,0,0,Property Damage,I,False,False,03,2,GROVE ST / MANILA AVE,,,,09061720__,0.45,07,,01,04,02,01,01,01,04,01,,AT,,I-78 / 12TH ST,,,,25,25,28,NaN,NaN,N,,3125,2020-01-10 07:42:00
80607,09,06,20-001983,HUDSON,JERSEY CITY,01,JERSEY CITY PD,EAST,0,0,0,0,Property Damage,I,False,False,02,2,GROVE ST / MANILA AVE,E,,,09061720__,0.45,07,,01,04,02,01,02,01,05,01,,AT,,I-78 / 12TH ST,,,,25,25,26,NaN,NaN,N,,3325,2020-01-27 06:30:00
80623,09,06,20-002071,HUDSON,JERSEY CITY,01,JERSEY CITY PD,EAST,0,0,0,0,Property Damage,B,False,False,06,2,GROVE ST / MANILA AVE,,,,09061720__,1.09,07,,01,04,02,01,07,01,04,01,50,FE,,BAY ST,,,,25,,28,NaN,NaN,N,,3335,2020-01-27 21:18:00
80657,09,06,20-002265,HUDSON,JERSEY CITY,01,JERSEY CITY PD,EAST,0,0,0,0,Property Damage,I,False,False,03,2,GROVE ST / MANILA AVE,,,,09061720__,0.45,07,,01,04,01,01,01,01,05,01,,AT,,I-78 / 12TH ST,,,,25,25,26,NaN,NaN,N,,3320,2020-01-30 07:47:00
80925,09,06,20-003397,HUDSON,JERSEY CITY,01,JERSEY CITY PD,EAST DISTRICT,0,1,0,1,Injury,I,False,False,13,1,GROVE ST / MANILA AVE,,,,09061720__,0.59,07,,01,04,02,02,01,02,05,01,,AT,,9TH ST,,,,25,25,26,NaN,NaN,N,,#280?,2020-02-13 10:41:00
81075,09,06,20-004028,HUDSON,JERSEY CITY,01,JERSEY CITY PD,EAST,0,0,0,0,Property Damage,B,False,False,01,2,GROVE ST / MANILA AVE,,,,09061720__,1.09,07,,01,04,01,01,01,01,05,01,100,FE,,BAY ST,,,,25,25,26,NaN,NaN,N,,3320,2020-02-21 09:30:00
81239,09,06,20-004710,HUDSON,JERSEY CITY,01,JERSEY CITY PD,EAST,0,0,0,0,Property Damage,I,False,False,02,2,GROVE ST / MANILA AVE,W,,,09061720__,0.34,07,,01,04,02,01,01,01,05,01,,AT,,I-78 / BOYLE PLAZA,,,,25,25,26,NaN,NaN,N,,2900,2020-02-29 12:21:00
81282,09,06,20-004923,HUDSON,JERSEY CITY,01,JERSEY CITY PD,EAST DISTRICT,0,0,0,0,Property Damage,B,False,False,02,2,GROVE ST / MANILA AVE,,,,09061720__,1.17,07,,01,04,02,01,01,01,04,01,100,FE,S,C COLUMBUS DR,,,,25,25,26,NaN,NaN,N,,2989,2020-03-03 07:56:00


In [5]:
gc = grove_crashes[['Date', 'Crash Location', 'Severity', 'SRI', 'MP']]
gc

,Date,Crash Location,Severity,SRI,MP
80141,2020-08-12 17:22:00,GROVE ST / MANILA AVE,Property Damage,09061720__,1.34
80262,2020-01-06 18:43:00,GROVE ST / MANILA AVE,Property Damage,09061720__,1.29
80313,2020-01-10 07:42:00,GROVE ST / MANILA AVE,Property Damage,09061720__,0.45
80607,2020-01-27 06:30:00,GROVE ST / MANILA AVE,Property Damage,09061720__,0.45
80623,2020-01-27 21:18:00,GROVE ST / MANILA AVE,Property Damage,09061720__,1.09
80657,2020-01-30 07:47:00,GROVE ST / MANILA AVE,Property Damage,09061720__,0.45
80925,2020-02-13 10:41:00,GROVE ST / MANILA AVE,Injury,09061720__,0.59
81075,2020-02-21 09:30:00,GROVE ST / MANILA AVE,Property Damage,09061720__,1.09
81239,2020-02-29 12:21:00,GROVE ST / MANILA AVE,Property Damage,09061720__,0.34
81282,2020-03-03 07:56:00,GROVE ST / MANILA AVE,Property Damage,09061720__,1.17


In [6]:
grove_crashes.Severity.value_counts()

Severity
Property Damage    27
Injury              7
Name: count, dtype: int64

In [7]:
gcll = sxs(gc, gc.apply(lambda r: get_mp_ll(r.SRI, r.MP), axis=1).rename('ll'))
gcll['LON'] = gcll.ll.apply(lambda p: p[0])
gcll['LAT'] = gcll.ll.apply(lambda p: p[1])
gcll = gcll.drop(columns='ll')
gcll

,Date,Crash Location,Severity,SRI,MP,LON,LAT
80141,2020-08-12 17:22:00,GROVE ST / MANILA AVE,Property Damage,09061720__,1.34,-74.044109,40.717517
80262,2020-01-06 18:43:00,GROVE ST / MANILA AVE,Property Damage,09061720__,1.29,-74.043745,40.718181
80313,2020-01-10 07:42:00,GROVE ST / MANILA AVE,Property Damage,09061720__,0.45,-74.041321,40.730150
80607,2020-01-27 06:30:00,GROVE ST / MANILA AVE,Property Damage,09061720__,0.45,-74.041321,40.730150
80623,2020-01-27 21:18:00,GROVE ST / MANILA AVE,Property Damage,09061720__,1.09,-74.042729,40.720957
80657,2020-01-30 07:47:00,GROVE ST / MANILA AVE,Property Damage,09061720__,0.45,-74.041321,40.730150
80925,2020-02-13 10:41:00,GROVE ST / MANILA AVE,Injury,09061720__,0.59,-74.041637,40.728102
81075,2020-02-21 09:30:00,GROVE ST / MANILA AVE,Property Damage,09061720__,1.09,-74.042729,40.720957
81239,2020-02-29 12:21:00,GROVE ST / MANILA AVE,Property Damage,09061720__,0.34,-74.041064,40.731774
81282,2020-03-03 07:56:00,GROVE ST / MANILA AVE,Property Damage,09061720__,1.17,-74.042939,40.719823


In [ ]:
fig = px.scatter_mapbox(
    gcll,
    lon='LON', lat='LAT',
    color='Severity',
    color_discrete_sequence=['yellow', 'orange', 'red'],
    hover_data=['Date', 'Crash Location', 'SRI', 'MP'],
    center=dict(lon=-74.042037, lat=40.725527),
    zoom=13.5,
    height=600,
)
legend_bgcolor = '50'
fig.update_layout(
    **mapbox,
    margin={"r":0,"t":0,"l":0,"b":0},
    title=dict(
        text="Grove St crashes (2020)",
        x=0.5, y=0.98,
        xanchor='center', yanchor='top',
        font=dict(size=32, color="white")
    ),
    legend=dict(
        title=dict(text=''),
        x=0.98, y=0.98,
        xanchor="right", yanchor="top",
        font=dict(
            size=14,
            color="white"
        ),
        bgcolor=f"rgba({legend_bgcolor},{legend_bgcolor},{legend_bgcolor},0.8)",
        bordercolor="white",
        borderwidth=2,
    ),
)
fig.show(config=dict(displayModeBar=False))